#### Publication Paper Analysis

From paper extract/generate:
1. title
2. author
3. publication date
4. publisher
5. summary

In [4]:
!pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [49]:
# Imports
import pandas as pd
import numpy as np
# PyPDF2 for loading PDF docs
import PyPDF2
import openai
from langchain.llms import OpenAI
import os
os.environ["OPENAI_API_KEY"] = ""
api_key = ""

In [16]:
pdfObj = open('../DGPdfs/01.pdf', 'rb')
pdfReader = PyPDF2.PdfReader(pdfObj)

# get number of pages in pdf file
print(pdfReader.pages)
data = pdfReader.metadata
print(data)

# create single page obj
singlePageObj = pdfReader.pages[0]
# extract content
# print(singlePageObj.extract_text())

pdfObj.close()

{'/Author': 'amd', '/CreationDate': 'D:20140528102257Z', '/Creator': 'PageMaker 7.0', '/ModDate': "D:20140528192024+05'30'", '/Producer': 'GPL Ghostscript 8.15', '/Title': 'AH1092.pmd', '/rgid': 'PB:262677268_AS:102063434371080@1401345227537'}


In [18]:
pdf1 = PyPDF2.PdfReader(open('../DGPdfs/01.pdf', 'rb'))

In [22]:
pdf1.metadata

{'/Author': 'amd',
 '/CreationDate': 'D:20140528102257Z',
 '/Creator': 'PageMaker 7.0',
 '/ModDate': "D:20140528192024+05'30'",
 '/Producer': 'GPL Ghostscript 8.15',
 '/Title': 'AH1092.pmd',
 '/rgid': 'PB:262677268_AS:102063434371080@1401345227537'}

In [28]:
pg1 = pdf1.pages[0].extract_text()

In [29]:
print(type(pg1))
print(pg1)

<class 'str'>
See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/262677268
Cultivation and Processing of Potato in Bihar : Issues and Strategies
Article  · Oct ober 2014
CITATIONS
6READS
2,775
4 author s, including:
Some o f the author s of this public ation ar e also w orking on these r elat ed pr ojects:
Agricult ural R eviews View pr oject
Crop Insur ance and its adoption  View pr oject
K. M. Singh
Dr R ajendr a Pr asad Centr al Agricult ural Univ ersity
473 PUBLICA TIONS    1,906  CITATIONS    
SEE PROFILE
Abhay K umar
ICAR R esearch Comple x for East ern R egion
151 PUBLICA TIONS    500 CITATIONS    
SEE PROFILE
All c ontent f ollo wing this p age was uplo aded b y K. M. Singh  on 29 May 2014.
The user has r equest ed enhanc ement of the do wnlo aded file.


In [13]:
# !pip install pdfx

In [12]:
import pdfx

# read pdf file
pdf_ = pdfx.PDFx("../DGPdfs/01.pdf")

# get metadata
pdf_.get_metadata()

{'Author': 'amd',
 'CreationDate': 'D:20140528102257Z',
 'Creator': 'PageMaker 7.0',
 'ModDate': "D:20140528192024+05'30'",
 'Producer': 'GPL Ghostscript 8.15',
 'Title': 'AH1092.pmd',
 'rgid': 'PB:262677268_AS:102063434371080@1401345227537',
 'rdf': {'li': 'AH1092.pmd'},
 'dc': {'format': 'application/pdf',
  'creator': ['amd'],
  'title': {'x-default': 'AH1092.pmd'}},
 'xap': {'CreateDate': '2014-05-28T10:22:57',
  'CreatorTool': 'PageMaker 7.0',
  'ModifyDate': '2014-05-28T19:20:24+05:30',
  'MetadataDate': '2014-05-28T19:20:24+05:30'},
 'pdf': {'Producer': 'GPL Ghostscript 8.15'},
 'xapmm': {'DocumentID': 'uuid:27fd8c55-6c12-42b6-b19f-184594a22baa',
  'InstanceID': 'uuid:b95723e2-ff04-48a8-b44e-3cf312798f63'},
 'Pages': 7}

In [27]:
llm = OpenAI(openai_api_key=api_key, temperature=0)

In [30]:
q = "When was this paper published?"
desc = llm(pg1 + q)

In [32]:
print(desc.strip())

This paper was published in October 2014.


In [33]:
q = "Who's the author of this paper?"
desc = llm(pg1 + q)

In [35]:
print(desc.strip())

The authors of this paper are K. M. Singh and Abhay Kumar.


In [36]:
q_t = "What is the title of this paper?"
title = llm(pg1 + q_t)

In [38]:
print(title.strip())

Cultivation and Processing of Potato in Bihar: Issues and Strategies


In [39]:
q_p = "Who is the publisher of this paper?"
publisher = llm(pg1 + q_p)

In [41]:
print(publisher.strip())

The publisher of this paper is the Indian Council of Agricultural Research (ICAR).


In [54]:
query_ = "Provide the following information from this paper: title, author, publisher, and date of publication"
info = llm(pg1 + query_)

In [56]:
print(info.strip())

Title: Cultivation and Processing of Potato in Bihar: Issues and Strategies
Author: K. M. Singh, Abhay Kumar
Publisher: ICAR Research Complex for Eastern Region
Date of Publication: October 2014


##### Summarize document using chatGPT model

In [50]:
def summarize_pdf(pdf_file_path):
    # Declare a string variable to hold the summary  
    summary = ""

    # Open file
    pdf_file = open(pdf_file_path, 'rb')
    # Read pdf file with PyPDF2
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Iterate over the pages in the PDF file
    for page_num in range(len(pdf_reader.pages)):
        # Extract the text from the page
        page_content = pdf_reader.pages[page_num].extract_text().lower()
        messages = [
            {"role": "system", "content": "You are a helpful research associate"},
            {"role": "user", "content": f"summarize this: {page_content}"},
        ]

        response = openai.ChatCompletion.create(
                        model = "gpt-3.5-turbo-0613",
                        messages = messages,
        )
        page_summary = response["choices"][0]["message"]["content"]
        summary = summary + page_summary + "\n"

    pdf_file.close()
    return summary

In [51]:
pdf_file_path = "../DGPdfs/01.pdf"

In [52]:
summary = summarize_pdf(pdf_file_path)

In [43]:
# Counting number of words in file

In [70]:
print(summary)

The publication titled "Cultivation and Processing of Potato in Bihar: Issues and Strategies" was published in October 2014 and has received 6 citations and 2,775 reads. It features the work of four authors, including K. M. Singh and Abhay Kumar. K. M. Singh has 473 publications and 1,906 citations, while Abhay Kumar has 151 publications and 500 citations. The content following the page was uploaded by K. M. Singh on May 29, 2014, and the user has requested enhancements to the downloaded file.
In this research article, the authors discuss the cultivation and processing of potato in Bihar, India. They highlight the importance of potato as a major food crop in Bihar and its ability to provide high yields and nutritional value. The authors emphasize the need for efficient marketing strategies to successfully diversify and commercialize potato production and increase farmers' income. They also discuss the challenges of enhancing productivity and quality in the face of shrinking arable land

In [71]:
q_ = "summarize this content"
summary_2 = llm(summary + q_)

In [73]:
# this is a summary of summary
print(summary_2.strip())

This research article discusses the cultivation and processing of potato in Bihar, India, highlighting the importance of potato as a major food crop and its ability to provide high yields and nutritional value. The authors emphasize the need for efficient marketing strategies to successfully diversify and commercialize potato production and increase farmers' income. They also discuss the challenges of enhancing productivity and quality in the face of shrinking arable land, reduced water availability, changing climate conditions, and various stresses. The authors suggest the establishment of new cold stores and processing industries to minimize transport costs and create employment opportunities. They also emphasize the need for increased utilization and export of potatoes to absorb excess production and sustain growth. The text also provides details on potato production in Bihar and discusses the opportunities, challenges, and strategies related to potato farming. It mentions the need 

##### Summarize paper using langchain

In [57]:
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [58]:
llm = OpenAI(temperature=0, openai_api_key=api_key)

In [64]:
from langchain.document_loaders import PyPDFLoader
import time

In [65]:
start_time = time.time()
loader = PyPDFLoader(pdf_file_path)
docs = loader.load_and_split()
chain = load_summarize_chain(llm, chain_type="map_reduce")
summary_ = chain.run(docs)
end_time = time.time()
elapsed_time = end_time - start_time

In [61]:
def summarize_pdf(pdf_file_path):
    loader = PyPDFLoader(pdf_file_path)
docs = loader.load_and_split()
chain = load_summarize_chain(llm, chain_type="map_reduce")
summary_ = chain.run(docs)
    pass

In [66]:
print(f"It took {elapsed_time} seconds to generate the summary")

It took 15.003280401229858 seconds to generate the summary


In [67]:
print(summary_)

 This article examines the cultivation and processing of potatoes in Bihar, India, and the strategies for improvement. It discusses the importance of potato as a short-duration crop with high yields and nutritional value, and the need for efficient marketing to ensure successful diversification and commercialization. It also looks at the challenges of shrinking arable land, reduced water availability, changing climatic conditions, and expanding biotic and abiotic stresses. The article concludes with a discussion of the current scenario, constraints, and possible solutions for enhancing potato cultivation in India and Bihar. It is based on research from the Directorate of Horticulture, Government of Bihar, and the book Global Supply Chains, Standards and the Poor by JFM Swnnen.
